In [2]:
import sys
import os
import numpy as np
import pandas as pd
#sys.path.append(os.path.dirname(os.path.dirname(os.path.dirname(os.path.abspath('"Top 10 Funds by AUM Positioning - Data Gen.ipynb"')))))
#sys.path.append(os.path.dirname(os.path.dirname(os.path.dirname(os.path.abspath('"Top 10 Funds by AUM Positioning - Data Gen.ipynb"')))))
import apis.pa.mypaengine as pa
import apis.qe.myqengine as qe

Python-dotenv could not parse statement starting at line 5
Python-dotenv could not parse statement starting at line 5


In [3]:
time_series = qe.TimeSeries(start_date = '0M',end_date = '0M')

univ = qe.ScreenUniverse(universe_expr = '(ISAV(FFD_SHR_CLS_AUM(0,ACT,ROLL,USD))=1 AND ISAV(P_PRICE(0,USD))=1 AND UPPERCASE(P_EXCOUNTRY(ISO2))=UPPERCASE("US") AND PROPER(FFD_CLASS_ASSET(TEXT))=PROPER("EQUITY") AND IF(UPPERCASE(FFD_BRAND)=UPPERCASE("DFA"),0,1)=1)=1')

req = qe.calculate(universe = univ, dates = time_series,formulas = ["FFD_NAME",'P_PRICE(0,USD)',"FFD_SHR_CLS_AUM(0,ACT,ROLL,USD)","LSS_TICKER","FFD_BMK_ID","FFD_BRAND"])

In [14]:
df = req.data.copy()
df = df.rename(columns = {"FFD_SHR_CLS_AUM(0,ACT,ROLL,USD)":'aum',
                       "LSS_TICKER":'lionshares_id',
                       "FFD_BMK_ID":'bench_id'})

df_append= pd.DataFrame()
bench_list = ['SP50','R.1000','R.1000G','R.1000V','R.2000','R.2000V','R.2000G','R.3000','R.MID','990300','892400','990100','899901','891800']
for bench in bench_list:
    df_append = pd.concat([df_append,df[df['bench_id']==bench].dropna().sort_values('aum',ascending=False).head(10)])

df_append['pa_port_id'] = "LION:"+df_append['lionshares_id']
df_append['pa_bench_id'] = "BENCH:"+df_append['bench_id']

df_append

,DATE,UNIVERSE,FFD_NAME,price,aum,lionshares_id,bench_id,FFD_BRAND,pa_port_id,pa_bench_id
2104,20230731,SWPPX,Schwab Capital Trust,70.88,7.506143e+10,SWPPX-US,SP50,Schwab,LION:SWPPX-US,BENCH:SP50
8416,20230731,TRJZX,Rowe T Price Growth Stock Fund Inc,85.45,2.139821e+10,TRJZX-US,SP50,T. Rowe Price,LION:TRJZX-US,BENCH:SP50
7113,20230731,HDGFX,Hartford Mutual Funds Inc,31.69,1.632903e+10,HDGFX-US,SP50,Hartford,LION:HDGFX-US,BENCH:SP50
5349,20230731,JUEMX,JPMorgan Trust I,20.64,1.404336e+10,JUEMX-US,SP50,JPMorgan,LION:JUEMX-US,BENCH:SP50
6408,20230731,PRUFX,Rowe T Price Growth Stock Fund Inc,84.72,1.350300e+10,PRUFX-US,SP50,T. Rowe Price,LION:PRUFX-US,BENCH:SP50
...,...,...,...,...,...,...,...,...,...,...
7818,20230731,TENWX,TIAA-CREF Funds,10.99,4.123690e+09,TENWX-US,891800,TIAA,LION:TENWX-US,BENCH:891800
8413,20230731,TRGZX,T. Rowe Price Emerging Markets Stock Fund,37.21,3.400960e+09,TRGZX-US,891800,T. Rowe Price,LION:TRGZX-US,BENCH:891800
5928,20230731,POEFX,Principal Funds Inc,10.11,2.775903e+09,POEFX-US,891800,Principal,LION:POEFX-US,BENCH:891800
1112,20230731,LZEMX,Lazard Funds Inc,17.19,2.481409e+09,LZEMX-US,891800,Lazard,LION:LZEMX-US,BENCH:891800


In [15]:
df_append.to_csv('../../data/Top Fund Positioning/fund_screen_20230731.csv')

In [9]:
pa_document_name = "PERSONAL:API_SP"
pa_component_category = "main / Weights"
pa_component_name = "Weights"

#Pass our to PA inputs to our custom class
pa_doc = pa.DocumentTemplate(
                    pa_document_name = pa_document_name,
                    pa_component_category = pa_component_category,
                    pa_component_name = pa_component_name,
                    )


In [40]:
#Define PA Inputs
temp_df =df_append[df_append['bench_id']=='SP50']
portfolios = temp_df['pa_port_id'].to_list()
benchmarks = temp_df['pa_bench_id'].to_list()
enddate = '0M'
pa_doc.run_calc(
               portfolios,
               benchmarks,
               end_date = enddate,
               curr = 'USD',
               mode= 'B&H'
              )
df_full = pd.DataFrame()
for x in list(pa_doc.data.keys()):
    temp = pa_doc.data[x][0]
    temp['port_x_bench'] = x
    df_full = pd.concat([df_full,temp])

In [18]:
pa_document_name = "PERSONAL:API_RUSSELL"
pa_component_category = "main / Weights"
pa_component_name = "Weights"

#Pass our to PA inputs to our custom class
pa_doc = pa.DocumentTemplate(
                    pa_document_name = pa_document_name,
                    pa_component_category = pa_component_category,
                    pa_component_name = pa_component_name,
                    )
portfolios = []
benchmarks = []

#set Russel benchmarks
rus_benchs = ['R.1000','R.1000G','R.1000V','R.2000','R.2000V','R.2000G','R.3000','R.MID']

#Define PA Inputs
temp_df = df_append[df_append['bench_id'].isin(rus_benchs)]
enddate = '0M'
#Run PA Calc
pa_doc.run_calc(
            temp_df['pa_port_id'].to_list(),
            temp_df['pa_bench_id'].to_list(),
            end_date = enddate,
            curr = 'USD',
            mode= 'B&H'
            )
#Append to dataframe
for x in list(pa_doc.data.keys()):
    temp = pa_doc.data[x][0]
    temp['port_x_bench'] = x
    df_full = pd.concat([df_full,temp])
        

In [19]:
pa_document_name = "PERSONAL:API_MSCI"
pa_component_category = "main / Weights"
pa_component_name = "Weights"

#Pass our to PA inputs to our custom class
pa_doc = pa.DocumentTemplate(
                    pa_document_name = pa_document_name,
                    pa_component_category = pa_component_category,
                    pa_component_name = pa_component_name,
                    )
msci_benchs = ['990300','892400','990100','899901','891800']

temp_df = df_append[df_append['bench_id'].isin(msci_benchs)]
enddate = '0M'
pa_doc.run_calc(
            temp_df['pa_port_id'].to_list(),
            temp_df['pa_bench_id'].to_list(),
            end_date = enddate,
            curr = 'USD',
            mode= 'B&H'
            )
for x in list(pa_doc.data.keys()):
    temp = pa_doc.data[x][0]
    temp['port_x_bench'] = x
    df_full = pd.concat([df_full,temp])

In [39]:
#Cleaning up data
df_full=df_full.replace('',np.nan)
df_full["GICS Sector"]= df_full["GICS Sector"].replace('--','Other')
df_full["GICS Sub Industry"]= df_full["GICS Sub Industry"].replace('--','Other')
df_full["GICS Industry Name"]= df_full["GICS Industry Name"].replace('--','Other')
df_full["GICS Industry Group"]= df_full["GICS Industry Group"].replace('--','Other')
df_full["Asset Type+(Client Definition/+FactSet)"]= df_full["Asset Type+(Client Definition/+FactSet)"].replace('--','Other')
df_full["Asset Class+(Client Definition/+FactSet)"]= df_full["Asset Class+(Client Definition/+FactSet)"].replace('--','Other')
df_full[['Port.+Weight','Bench.+Weight','Difference']] = df_full[['Port.+Weight','Bench.+Weight','Difference']].fillna(0)
df_full[['Port.+Weight','Bench.+Weight','Difference']] =df_full[['Port.+Weight','Bench.+Weight','Difference']].astype(float)

pattern = r'LION:(.*?)_x_'
df_full['lionshares_id'] = df_full['port_x_bench'].str.extract(pattern, expand=False)

df_full=df_full.drop(columns = 'portfolio')

df_full= df_full.set_index(['lionshares_id','Symbol']).join(df.set_index('lionshares_id'),how='left',on='lionshares_id')

In [36]:

df_full = df_full.rename(columns ={'Asset Type+(Client Definition/+FactSet)':'Asset Type',
                        'Asset Class+(Client Definition/+FactSet)':'Asset Class',
                        'price':'Fund Price',
                        'security0':'Security Name',
                        'Port.+Weight':'Portfolio Weight',
                        'Bench.+Weight':'Benchmark Weight',
                        'Difference':'Active Weight',
                        'UNIVERSE':'Fund Ticker',
                        'FFD_BRAND':'Fund Brand'
                        })

df_full = df_full.drop(columns = ['port_x_bench'])

In [37]:
df_full.to_pickle('../../data/Top Fund Positioning/t10_full_universe_weights_20230731.pkl')